In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

Entities_Embedding_file = '/kaggle/input/datadrugtarget/results_3mai2023/e/dict_e.list'
Entities_Embedding_List = {}
IdEntiy__Embedding_List = {}
proteinidlist=[]
proteinnamelist=[]
drugnamelist=[]
drugidlist=[]
with open(Entities_Embedding_file) as f:
    for line in f:
        it = line.strip().split("\t")
        id = int(it[0])
        entity = str(it[1])
        if id > 345:
            proteinidlist.append(id)
            proteinnamelist.append(entity)
            if entity not in Entities_Embedding_List:
                Entities_Embedding_List[entity] = set()
            Entities_Embedding_List[entity].add(int(id))
            if id not in IdEntiy__Embedding_List:
                IdEntiy__Embedding_List[id] = set()
            IdEntiy__Embedding_List[id].add(str(entity))
        else:
            drugidlist.append(id)
            drugnamelist.append(entity)
print(len(proteinnamelist))


        



In [ ]:
# Load entity embeddings
entity_embeddings = np.loadtxt('/kaggle/input/datadrugtarget/results_3mai2023/e/entity_embedding_dt_e.csv')

# Your other code here...

# Assuming embedcosinusDrugslist, embedcosinusProteinslist, and embedcosinusTargetlist 
# contain the indices of drugs, proteins, and targets respectively
subembedding=entity_embeddings[proteinidlist]
print(len(subembedding))
print(subembedding)
from sklearn.decomposition import PCA
pca = PCA(n_components=90)
loaded_embedding_e_md_pca = pca.fit_transform(subembedding)
#combined_embeddings = np.concatenate((loaded_embedding_prot_e_protbert_md_pca, subembedding), axis=0)
#print("combined_embeddings.shape()")
#print(combined_embeddings.shape)
subembeddingdrugs=entity_embeddings[drugidlist]
subembeddingdrugs_md= pca.fit_transform(subembeddingdrugs)
combined_embeddings = np.concatenate((subembeddingdrugs_md, loaded_embedding_e_md_pca), axis=0)
print("combined_embeddings.shape()")
print(combined_embeddings.shape)
np.savetxt(
        os.path.join('/kaggle/working/entity_embedding_dt_e_withKGE.csv'),
        combined_embeddings
    )
similarities_proteins = cosine_similarity(loaded_embedding_e_md_pca)


# Now similarities_proteins contains cosine similarities between proteins


**Combination with Protbert Embedding******

import numpy as np
loaded_ids_prot_e_protbert = np.load('/kaggle/input/protbert-gpcr-embedding/protein_gpcr_ids.npy')
# Extract UniProt identifiers using list comprehension
uniport_identifiers = [entry.split('|')[1] for entry in loaded_ids_prot_e_protbert]
print(len(uniport_identifiers))

loaded_embedding_prot_e_protbert = np.load('/kaggle/input/protbert-gpcr-embedding/protbert_gpcr_embeddings.npy')
# Index of the vector you want to delete
index_to_delete = 1
print("loaded_embedding_prot_e_protbert.shape()")
print(loaded_embedding_prot_e_protbert.shape)
from sklearn.decomposition import PCA
desired_components = 90

# Apply PCA to loaded_embedding_prot_e_protbert_md
pca = PCA(n_components=90)
loaded_embedding_prot_e_protbert_md_pca = pca.fit_transform(loaded_embedding_prot_e_protbert)
#combined_embeddings = np.concatenate((loaded_embedding_prot_e_protbert_md_pca, subembedding), axis=0)
#print("combined_embeddings.shape()")
#print(combined_embeddings.shape)
subembeddingdrugs=entity_embeddings[drugidlist]
subembeddingdrugs_md= pca.fit_transform(subembeddingdrugs)
combined_embeddings = np.concatenate((subembeddingdrugs_md, loaded_embedding_prot_e_protbert_md_pca), axis=0)
print("combined_embeddings.shape()")
print(combined_embeddings.shape)
np.savetxt(
        os.path.join('/kaggle/working/entity_embedding_dt_gpcr_withprotbert.csv'),
        combined_embeddings
    )
similarities_proteins = cosine_similarity(loaded_embedding_prot_e_protbert_md_pca)

In [ ]:
# Save cosine similarities to a text file
with open('/kaggle/working/cosine_similarities_gpcr_proteins.txt', 'w') as f:
    # Save drug similarities
    for i, row in enumerate(similarities_proteins):
        for j, similarity in enumerate(row):
            protein1 = proteinnamelist[i]
            protein2 = proteinnamelist[j]
            f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity))

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
similarities_protein_ECLD = euclidean_distances(entity_embeddings[proteinidlist])
# Save cosine similarities to a text file
with open('/kaggle/working/eucludian_similarities_gpcr_proteins.txt', 'w') as f:
    # Save drug similarities
    for i, row in enumerate(similarities_protein_ECLD):
        for j, similarity in enumerate(row):
            protein1 = proteinnamelist[i]
            protein2 = proteinnamelist[j]
            f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity))



In [ ]:
from sklearn.metrics.pairwise import manhattan_distances
similarities_proteins_mnht = manhattan_distances(entity_embeddings[proteinidlist])
# Save cosine similarities to a text file
with open('/kaggle/working/manhatan_similarities_gpcr_proteins.txt', 'w') as f:
    # Save drug similarities
    for i, row in enumerate(similarities_proteins_mnht):
        for j, similarity in enumerate(row):
            protein1 = proteinnamelist[i]
            protein2 = proteinnamelist[j]
            f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity))


In [ ]:
from scipy.stats import pearsonr
similarities_proteins_pearson = np.array([[pearsonr(a, b)[0] for b in entity_embeddings[proteinidlist]] for a in entity_embeddings[proteinidlist]])
# Save cosine similarities to a text file
with open('/kaggle/working/pearson_similarities_gpcr_proteins.txt', 'w') as f:
    # Save drug similarities
    for i, row in enumerate(similarities_proteins_pearson):
        for j, similarity in enumerate(row):
            protein1 = proteinnamelist[i]
            protein2 = proteinnamelist[j]
            f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity))

from scipy.stats import pearsonr
similarities_proteins = np.array([[pearsonr(a, b)[0] for b in entity_embeddings[proteinidlist]] for a in entity_embeddings[proteinidlist]])
# Save cosine similarities to a text file
with open('/kaggle/working/pearsonr_similarities_e_proteins.txt', 'w') as f:
    # Save drug similarities
    for i, row in enumerate(similarities_proteins_mnht):
        for j, similarity in enumerate(row):
            protein1 = proteinnamelist[i]
            protein2 = proteinnamelist[j]
            f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity))


import os
import numpy as np
from scipy.stats import pearsonr
from multiprocessing import Pool

# Get the number of available CPU cores
num_processes = os.cpu_count()
print("num_processes")
print(num_processes)

# Assuming entity_embeddings is a NumPy array
def compute_similarity(i):
    similarities = []
    for j in range(len(entity_embeddings)):
        similarity = pearsonr(entity_embeddings[i], entity_embeddings[j])[0]
        similarities.append((i, j, similarity))
    return similarities


pool = Pool(processes=num_processes)

# Compute Pearson similarities using multiprocessing
results = pool.map(compute_similarity, range(len(entity_embeddings)))
pool.close()
pool.join()

# Flatten the list of similarities from different processes
similarities_proteins = [similarity for sublist in results for similarity in sublist]

# Save Pearson similarities to a text file
with open('/kaggle/working/pearsonr_similarities_ic_drugs.txt', 'w') as f:
    # Save protein similarities
    for similarity in similarities_proteins:
        protein1, protein2, similarity_value = similarity
        f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity_value))

print("End pearson embedding")


In [ ]:
from sklearn.metrics import jaccard_score
# Threshold for converting numerical values to binary (0 or 1)
threshold = 0.5  # You can adjust this threshold based on your data

# Convert numerical values to binary based on the threshold
binary_protein_sets = [[int(value > threshold) for value in entity_embeddings[proteinid]] for proteinid in proteinidlist]

# Calculate Jaccard similarities between proteins
similarities_proteins_jaccard = np.array([[jaccard_score(set_a, set_b) for set_b in binary_protein_sets] for set_a in binary_protein_sets])
with open('/kaggle/working/jaccard_similarities_gpcr_proteins.txt', 'w') as f:
    # Save drug similarities
    for i, row in enumerate(similarities_proteins_jaccard):
        for j, similarity in enumerate(row):
            protein1 = proteinnamelist[i]
            protein2 = proteinnamelist[j]
            f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity))

In [ ]:
# Assuming embedcosinusDrugslist, embedcosinusProteinslist, and embedcosinusTargetlist
# contain the indices of drugs, proteins, and targets respectively
subembedding=combined_embeddings[drugidlist]
print(len(subembedding))
print(subembedding)
# Calculate cosine similarities between proteins
similarities_drugs = cosine_similarity(subembedding)

# Save cosine similarities to a text file
with open('/kaggle/working/cosine_similarities_gpcr_drugs.txt', 'w') as f:
    # Save drug similarities
    for i, row in enumerate(similarities_drugs):
        for j, similarity in enumerate(row):
            protein1 = drugnamelist[i]
            protein2 = drugnamelist[j]
            f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity))

from sklearn.metrics.pairwise import euclidean_distances
similarities_protein_ECLD = euclidean_distances(entity_embeddings[drugidlist])
# Save cosine similarities to a text file
with open('/kaggle/working/eucludian_similarities_gpcr_drugs.txt', 'w') as f:
    # Save drug similarities
    for i, row in enumerate(similarities_protein_ECLD):
        for j, similarity in enumerate(row):
            protein1 = drugnamelist[i]
            protein2 = drugnamelist[j]
            f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity))

from sklearn.metrics.pairwise import manhattan_distances
similarities_proteins_mnht = manhattan_distances(entity_embeddings[drugidlist])
# Save cosine similarities to a text file
with open('/kaggle/working/manhatan_similarities_gpcr_drugs.txt', 'w') as f:
    # Save drug similarities
    for i, row in enumerate(similarities_proteins_mnht):
        for j, similarity in enumerate(row):
            protein1 = drugnamelist[i]
            protein2 = drugnamelist[j]
            f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity))

from scipy.stats import pearsonr
similarities_proteins_pearson = np.array([[pearsonr(a, b)[0] for b in entity_embeddings[drugidlist]] for a in entity_embeddings[drugidlist]])
# Save cosine similarities to a text file
with open('/kaggle/working/pearson_similarities_gpcr_drugs.txt', 'w') as f:
    # Save drug similarities
    for i, row in enumerate(similarities_proteins_pearson):
        for j, similarity in enumerate(row):
            protein1 = drugnamelist[i]
            protein2 = drugnamelist[j]
            f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity))

from sklearn.metrics import jaccard_score
# Threshold for converting numerical values to binary (0 or 1)
threshold = 0.5  # You can adjust this threshold based on your data

# Convert numerical values to binary based on the threshold
binary_protein_sets = [[int(value > threshold) for value in entity_embeddings[proteinid]] for proteinid in drugidlist]

# Calculate Jaccard similarities between proteins
similarities_proteins_jaccard = np.array([[jaccard_score(set_a, set_b) for set_b in binary_protein_sets] for set_a in binary_protein_sets])
with open('/kaggle/working/jaccard_similarities_gpcr_drugs.txt', 'w') as f:
    # Save drug similarities
    for i, row in enumerate(similarities_proteins_jaccard):
        for j, similarity in enumerate(row):
            protein1 = drugnamelist[i]
            protein2 = drugnamelist[j]
            f.write('{}\t{}\t{}\n'.format(protein1, protein2, similarity))